# Resumen de las sentencias empleadas en el curso para Agentes

### Instalar librerías necesarias

In [ ]:
!pip install crewai -Uq # Intalla la libreria CrewAI, que a su vez necesita de LangChain
!pip install beautifulsoup4 -q # Instala la libreria para hacer Scrapping
!pip install typing -q # Instala la libreria para hacer Typing

## Ejemplos

### Primer ejemplo, investigador y redactor

In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# Define modelo a utilizar
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Define agentes con roles y metas
investigador = Agent(
    role = 'Analista Investigador Senior',
    goal = 'Descubrir desarrollos de vanguardia en IA y Ciencias de Datos',
    backstory = """Trabajas en una destacada empresa de tecnología.
    Tu experiencia radica en identificar tendencias emergentes.
    Tienes un talento especial para diseccionar datos complejos y presentar
    perspectivas accionables.""",
    verbose = True,
    allow_delegation = False,
    llm = llm
)

escritor = Agent(
    role = 'Estratega de contenidos sobre tecnología',
    goal = 'Crear contenido convincente sobre avances tecnológicos',
    backstory = """Eres un estratega de contenido renombrado, conocido por tus
    artícuños perpicaces y atractivos. Transformas conceptos complejos en
    narrativas convincentes.""",
    verbose = True,
    allow_delegation = False,
    llm = llm
)

# Crea tareas para los agentes
tarea1 = Task(
    description = """Realiza un análisis exhaustivo e los últimos avances en IA
    en 2024. Identifica las tendencias clave, las tecnologías revolucionarias
    y los posibles impactos en la industria.""",
    expected_output=' Tu respuesta final debe ser un informe de análisis completo.',
    agent = investigador
)

tarea2 = Task(
    description ="""Utilizando los conocimientos proporcionados, desarrolla una
    publicación de blog atractiva que resalte los avances más significativos
    en IA. Tu publicación debe ser informativa pero accesible, dirigida a una
    audiencia con conocimientos tecnológicos. Haz que suene genial, evita
    palabras complejas para que no suene como IA.""",
    expected_output='Tu respuesta final debe ser la publicación completa del blog de al menos 4 párrafos.',
    agent = escritor
)

# Instancia el equipo con un proceso secuencial
equipo = Crew(
    agents=[investigador, escritor],
    tasks=[tarea1, tarea2],
    verbose=True,
    process=Process.sequential
)

# Pone el equipo a trabajar
resultado = equipo.kickoff()
print(resultado)

### Segundo ejemplo, equipo de redes sociales

In [ ]:
import requests
from bs4 import BeautifulSoup
from crewai import Agent, Task, Crew
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

# Define el modelo a utilizar
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Definimos la herramienta que vamos a utilizar
# Busca en internet con Tavily, procesa el HTML con Beautifulsoup
# retun_direct=False significa que esta herramienta retornará los resultados de manera privada al agente, y no lo hará públicamente a la aplicación
@tool("process_search_tool", return_direct=False)
def process_search_tool(url: str) -> str:
    """Procesa el contenido encontrado en Internet."""
    response = requests.get(url=url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text()

tools = [TavilySearchResults(max_results=1), process_search_tool]


# Crea los agentes
investigador_online = Agent(
    role="Investigador en Línea",
    goal="Investigar el tema en línea",
    backstory="""Tu rol principal es funcionar como un asistente de investigación en línea inteligente, hábil en buscar las últimas y más relevantes historias relacionadas a deportes. Posees la capacidad de acceder a una amplia gama de fuentes de noticias en línea, blogs y plataformas de redes sociales para recopilar información en tiempo real.""",
    verbose=True,
    allow_delegation=True,
    tools=tools,
    llm=llm
)

gestor_de_blog = Agent(
    role="Gestor de Blog",
    goal="Escribir el artículo del blog",
    backstory="""Eres un Gestor de Blog. El rol de un Gestor de Blog abarca varias responsabilidades críticas destinadas a transformar borradores iniciales en artículos de blog pulidos y optimizados para SEO que atraigan y hagan crecer a una audiencia. Partiendo de los borradores proporcionados por los investigadores en línea, el Gestor de Blog debe comprender a fondo el contenido, asegurándose de que esté alineado con el tono del blog, la audiencia objetivo y los objetivos temáticos. Las responsabilidades clave incluyen:

    1. Mejora de Contenido: Elevar la calidad del borrador mejorando la claridad, fluidez y atractivo. Esto implica refinar la narrativa, agregar encabezados atractivos y asegurarse de que el artículo sea fácil de leer e informativo.

    2. Optimización SEO: Implementar las mejores prácticas de optimización para motores de búsqueda. Esto incluye investigación e integración de palabras clave, optimización de descripciones meta y asegurarse de que las estructuras de URL y las etiquetas de encabezado mejoren la visibilidad en los resultados de búsqueda.

    3. Cumplimiento y Mejores Prácticas: Asegurarse de que el contenido cumpla con las normas legales y éticas, incluyendo las leyes de derechos de autor y la veracidad en la publicidad. El Gestor de Blog también debe mantenerse al día con las estrategias de SEO en evolución y las tendencias de blogs para mantener y mejorar la efectividad del contenido.

    4. Supervisión Editorial: Trabajar estrechamente con escritores y colaboradores para mantener una voz y calidad consistentes en todas las publicaciones del blog. Esto también puede implicar la gestión de un calendario de contenido, programar publicaciones para un compromiso óptimo y coordinar con los equipos de marketing para apoyar actividades promocionales.

    5. Análisis e Integración de Retroalimentación: Revisar regularmente las métricas de rendimiento para comprender el compromiso y las preferencias de la audiencia. Usar estos datos para refinar el contenido futuro y optimizar la estrategia general del blog.

    En resumen, el Gestor de Blog juega un papel fundamental en la conexión entre la investigación inicial y la publicación final, mejorando la calidad del contenido, asegurando la compatibilidad con SEO y alineándose con los objetivos estratégicos del blog. Esta posición requiere una combinación de habilidades creativas, técnicas y analíticas para gestionar y hacer crecer exitosamente la presencia del blog en línea.""",
    verbose=True,
    allow_delegation=True,
    tools=tools,
    llm=llm
)

gestor_de_redes_sociales = Agent(
    role="Gestor de Redes Sociales",
    goal="Escribir un tweet",
    backstory="""Eres un Gestor de Redes Sociales. El rol de un Gestor de Redes Sociales, particularmente para gestionar contenido en Twitter, implica transformar borradores de investigación en tweets concisos y atractivos que resuenen con la audiencia y se adhieran a las mejores prácticas de la plataforma. Al recibir un borrador de un investigador en línea, el Gestor de Redes Sociales tiene varias funciones críticas:

    1. Condensación de Contenido: Destilar el mensaje central del borrador en un tweet, que típicamente permite solo 280 caracteres. Esto requiere un enfoque agudo en la brevedad mientras se mantiene la esencia y el impacto del mensaje.

    2. Optimización de Compromiso: Crear tweets para maximizar el compromiso. Esto incluye el uso estratégico de un lenguaje atractivo, hashtags relevantes y temas oportunos que resuenen con la audiencia objetivo.

    3. Cumplimiento y Mejores Prácticas: Asegurarse de que los tweets sigan las directrices y mejores prácticas de Twitter, incluyendo el uso apropiado de menciones, hashtags y enlaces. También, adherirse a normas éticas, evitando la desinformación y respetando las normas de derechos de autor.

    En resumen, el rol del Gestor de Redes Sociales es crucial para aprovechar Twitter para difundir información de manera efectiva, interactuar con los seguidores y construir la presencia de la marca en línea. Esta posición combina habilidades de comunicación creativa con planificación estratégica y análisis para optimizar el impacto en las redes sociales.""",
    verbose=True,
    allow_delegation=True,
    tools=tools,
    llm=llm
)

gestor_de_marketing_de_contenido = Agent(
    role="Gestor de Marketing de Contenido",
    goal="Gestionar el equipo de marketing de contenido",
    backstory="""Eres un excelente Gestor de Marketing de Contenido. Tu rol principal es supervisar cada publicación del 'gestor de blog' y los tweets escritos por el 'gestor de redes sociales' y aprobar el trabajo para su publicación. Examina el trabajo y regula el lenguaje violento, el contenido abusivo y el contenido racista.

    Capacidades:

    Revisión Editorial: Analizar los borradores finales del gestor de blog y del gestor de redes sociales para la consistencia de estilo, alineación temática y flujo narrativo general.

    Garantía de Calidad: Realizar verificaciones detalladas de la precisión gramatical, la corrección factual y la adherencia a los estándares periodísticos en el contenido de noticias, así como la creatividad y efectividad en los anuncios.

    Retroalimentación: Proporcionar retroalimentación constructiva tanto al gestor de blog como al gestor de redes sociales, facilitando un entorno colaborativo para la mejora continua en la creación y presentación de contenido.""",
    verbose=True,
    allow_delegation=True,
    tools=tools,
    llm=llm
)

# Define las tareas
tarea1 = Task(
    description="""Escríbeme un informe sobre el partido de Argentina y Chile en la Copa América. Después de la investigación sobre los resultados de la primera fecha de la Copa América, pasa los hallazgos al gestor de blog para generar el artículo final del blog. Una vez hecho, pásalo al gestor de redes sociales para que escriba un tweet sobre el tema.""",
    expected_output="Informe sobre los resultados de la primera fecha de la Copa América",
    agent=investigador_online
)

tarea2 = Task(
    description="""Usando los hallazgos de la investigación del corresponsal de noticias, escribe un artículo para el blog. La publicación debe contener enlaces a las fuentes indicadas por el investigador en línea. Tu respuesta final DEBE ser el artículo completo del blog de al menos 3 párrafos.""",
    expected_output="Artículo del Blog",
    agent=gestor_de_blog
)

tarea3 = Task(
    description="""Usando los hallazgos de la investigación del corresponsal de noticias, escribe un tweet. Tu respuesta final DEBE ser el tweet completo.""",
    expected_output="Tweet",
    agent=gestor_de_redes_sociales
)

tarea4 = Task(
    description="""Revisar meticulosamente y armonizar el resultado final tanto del gestor de blog como del gestor de redes sociales, asegurando coherencia y excelencia en la publicación final. Una vez hecho, publica el informe final.""",
    expected_output="Informe Final",
    agent=gestor_de_marketing_de_contenido
)

# Define el equipo
agents = [investigador_online, gestor_de_blog, gestor_de_redes_sociales, gestor_de_marketing_de_contenido]

crew = Crew(
    agents=agents,
    tasks=[tarea1, tarea2, tarea3, tarea4],
    verbose=2,
    process=Process.sequential
)

# Pone el equipo a trabajar
resultado = crew.kickoff()
print(resultado)